# Chamorro Lexicon Expander

**Chamorro Lexicon Expander** is a Python project designed to expand the Chamorro-English dictionary by generating all possible affixed variations of Chamorro root words. This tool automates the process of creating word forms using common Chamorro prefixes, suffixes, and infixes according to linguistic rules. This project enables a more comprehensive representation of Chamorro vocabulary for language learners, linguists, and dictionary development.

**Name:** Schyuler Lujan <br>
**Date Started:** 10-Nov-2024 <br>
**Date Complete:** In Progress <br>

In [1]:
# Import libraries
import re
import pandas as pd
import csv

# Import and Clean Data

**About this data:** For this project, we will be using the words and part-of-speech tags from the Revised Chamorro-English dictionary, which is available for free at https://natibunmarianas.org/chamorro-dictionary/. We will be using this data because it is currently the only freely available resource online with the most complete and reliably accurate part-of-speech tags on Chamorro words. Part-of-speech tags will determine which words can be transformed with the different affixes.

In [21]:
# Import files and convert to dataframes
# tverbs_df = pd.read_csv("transitive-verbs.csv", encoding="utf-8")
# iverbs_df = pd.read_csv("intransitive-verbs.csv", encoding="utf-8")

In [22]:
# Preview dataframe
#tverbs_df.head()

# VOWEL HARMONY

**About Vowel Harmony:** In Chamorro, vowel harmony refers to how certain vowels must change into different vowels, depending upon the vowel that comes before it. In general, the vowels "å", "o", and "e" must shift if they are preceded by an "i".

## Vowel Harmony Lists

In [12]:
# Create a list of vowels to search for in the words - to be used for infixes and some prefixes
vowels = ['a', 'á', 'å', 'e', 'é', 'i', 'í', 'o', 'ó', 'u']

In [59]:
# Create a dictionary of vowel harmony rules - to be used for infixes
vowel_harmony_rules = {"å": "a", "o": "e", "u": "i"}

# Create a dictionary of vowel lowering / vowel fronting rules to be used with some prefixes
### DICTIONARY FOR VOWEL LOWERING RULES GOES HERE ###

## Apply Vowel Harmony Transformation

In [24]:
def vowel_harmonized_word(word, vowels, vowel_harmony_rules):
    """
    Transforms a word to apply vowel harmony rules and returns the vowel harmonized word
    """
    for letter in word:
        if letter in vowels:
            
            # If the first vowel is in the vowel harmony rule list, transform the word
            if letter in vowel_harmony_rules: 
                new_word = word.replace(letter, vowel_harmony_rules[letter], 1)
            else:
                new_word = word
            break # Stop iteration after first vowel
    
    return new_word

In [16]:
### DELETE ME TEST CODE ###
word = "chalåni"
vowel_harmonized_word(word, vowels, vowel_harmony_rules)

'chalåni'

## Vowel Harmonized Word Lists

In [60]:
def vowel_harmonized_words(data, vowels, vowel_harmony_rules):
    """
    Takes a list of words and transforms them with vowel harmony.
    Produces a list of tuples, with each tupple containing the new word, base word, and the transformation name.
    """
    # Initialize list to store new words
    new_words = []
    
    # Iterate through data and apply vowel harmony rules
    for word in data:
        new_word = vowel_harmonized_word(word, vowels, vowel_harmony_rules)
        new_words.append((new_word, word, "Vowel Harmony"))
        
    return new_words

In [61]:
### DELETE ME TEST CODE ###
word_list = ["chålan", "guma'", "to'lang"]
print(vowel_harmonized_words(word_list, vowels, vowel_harmony_rules))

[('chalan', 'chålan', 'Vowel Harmony'), ("gima'", "guma'", 'Vowel Harmony'), ("te'lang", "to'lang", 'Vowel Harmony')]


# INFIXES

**About Chamorro Infixes:** Infixes are affixes that occur within the word, rather than being attached in front of the word or at the end. In Chamorro, infixes are always applied before the first vowel of the word they are attached to. If the word it's being attached to starts with a vowel, the infix is still placed in front of that vowel. There are two infixes in Chamorro: -in- and -um-.

## Apply Infix Transformation

In [6]:
def infixed_word(word, vowels, infix):
    """
    Transforms a word with an infix and returns the infixed word.
    Vowel harmony transformations are in the function `vowel_harmonized_word`
    """
    
    for letter in word:
        if letter in vowels:
            i = word.index(letter) # Get vowel's index
            new_word = word[0:i]+infix+word[i:] # Add the infix before vowel
            break # Stop iteration after first vowel
    
    return new_word

In [7]:
### DELETE ME TEST CODE ###
word = "påtgon"
infix = "in"
infixed_word(word, vowels, infix)

'pinåtgon'

## Apply UM Infix Transformation

In [8]:
def apply_um_infix(data, vowels):
    """
    Iterates through a list of words and applies the UM Infix
    """
    # Define our infix
    infix = "um"
    
    # Intialize list to store the infixed word, root word, and the infix name
    new_words = []
    
    # Transform word with UM Infix and append the new word to new_words
    for word in data:
        new_word = infixed_word(word, vowels, infix)
        new_words.append((new_word, word, f"{infix} infix"))
        
    return new_words

In [9]:
### DELETE ME TEST CODE ###
data = ["hånao", "konne'", "taitai", "tufong"]
print(apply_um_infix(data, vowels))

[('humånao', 'hånao', 'um infix'), ("kumonne'", "konne'", 'um infix'), ('tumaitai', 'taitai', 'um infix'), ('tumufong', 'tufong', 'um infix')]


## Apply IN Infix Transformation

In [17]:
def apply_in_infix(data, vowels, vowel_harmony_rules):
    """
    Iterates through a list of words and applies the IN Infix.
    Vowel harmony rules must also be applied with the IN infix transformation.
    """
    # Define the infix
    infix = "in"
    
    # Intialize list to store the infixed word, root word, and the infix name
    new_words = []
    
    # Transform word with IN infix and append the new word to new_words
    for word in data:
        # Apply vowel harmony rules first
        harmonized_word = vowel_harmonized_word(word, vowels, vowel_harmony_rules)
        # Transform the harmonized_word with the IN infix
        new_word = infixed_word(harmonized_word, vowels, infix)
        new_words.append((new_word, word, f"{infix} infix"))
        
    return new_words

In [18]:
### DELETE ME TEST CODE ###
data = ["hånao", "konne'", "taitai", "tufong"]
print(apply_in_infix(data, vowels, vowel_harmony_rules))

[('hinanao', 'hånao', 'in infix'), ("kinenne'", "konne'", 'in infix'), ('tinaitai', 'taitai', 'in infix'), ('tinifong', 'tufong', 'in infix')]


# PREFIXES

**About Chamorro Prefixes:** Prefixes in Chamorro are attached to the front of a base word, and one or more prefixes can be attached. Some prefixes require morphophonemic changes to the base word when attached.

## Prefix Lists

We will create different lists of prefixes that do not require morphophonemic changes to the base word. Lists are organized by the type of base word they can be attached to. The prefix lists will include single prefixes and known combinations of prefixes.

In [34]:
# Create a list of prefixes for transitive verbs
transitive_verb_prefixes = [
    "án", "fa'", "á", "umá", "maná", "acha", "ma'acha", "fanma'acha", 
    "ké", "kéke",  "na'", "muna'", "nina'", "mannina'", "fanina'", "manna'", "fanna'",
    "ma", "manma", "fanma" 
]

# Create a list of prefixes for nouns
noun_prefixes = ["gai", "tai", "an", "mi", "é", "fa'", "mama'", "fama'", "ga'", "pinat"]

# Create a list of prefixes for adjectives
adjective_prefixes = ["chat", "fa'", "há", "lá", "pinat", "na'", "ná", "acha", "na'lá"]

# Create a list of the directional prefixes
directional_prefixes = ["gé", "hat", "san", "tak", "talak", "ya"]

In [23]:
### DELETE ME: PREFIX NOTES ###
# combos é+um, é+man, ga+um+verb, ga+indef man-, ha+ma+trans verb
# vowel lowering or harmony required for fa', ge', hat, san, tak, talak
# vowel harmony required for gai, tai, mi

## Apply Prefix Transformation

In [50]:
def apply_prefix(word, prefix, vowels):
    """
    Transform a word by attaching a prefix to the front, and returning the new, transformed word.
    """
    # Apply prefix
    if word[0] in vowels and prefix[-1] in vowels:
        new_word = prefix + """'""" + word
    else:
        new_word = prefix + word
    
    return new_word

In [62]:
### DELETE ME TEST CODE ###
word = "li'e'"
prefix = "man"
print(apply_prefix(word, prefix, vowels))

manli'e'


## Apply Prefix Transformations - Without Morphophonemic Changes

In [55]:
def apply_prefixes_no_morphophonemic_change(data, prefixes, vowels):
    """
    Iterates through a list of words and prefixes and attaches each prefix to the front of each word.
    Returns a list of tuples, where each tuple is the new word, the root word, and the prefix.
    """
    # Initialize list to store new words
    new_words = []
    
    # Apply prefixes
    for prefix in prefixes:
        for word in data:
            new_word = apply_prefix(word, prefix, vowels)
            new_words.append((new_word, word, f"{prefix} prefix"))
            
    return new_words

In [57]:
### DELETE ME TEST CODE ###
word_list = ["kånno'", "oppe'"]
print(apply_prefixes_no_morphophonemic_change(word_list, transitive_verb_prefixes, vowels))

[("ánkånno'", "kånno'", 'án prefix'), ("ánoppe'", "oppe'", 'án prefix'), ("fa'kånno'", "kånno'", "fa' prefix"), ("fa'oppe'", "oppe'", "fa' prefix"), ("ákånno'", "kånno'", 'á prefix'), ("á'oppe'", "oppe'", 'á prefix'), ("umákånno'", "kånno'", 'umá prefix'), ("umá'oppe'", "oppe'", 'umá prefix'), ("manákånno'", "kånno'", 'maná prefix'), ("maná'oppe'", "oppe'", 'maná prefix'), ("achakånno'", "kånno'", 'acha prefix'), ("acha'oppe'", "oppe'", 'acha prefix'), ("ma'achakånno'", "kånno'", "ma'acha prefix"), ("ma'acha'oppe'", "oppe'", "ma'acha prefix"), ("fanma'achakånno'", "kånno'", "fanma'acha prefix"), ("fanma'acha'oppe'", "oppe'", "fanma'acha prefix"), ("kékånno'", "kånno'", 'ké prefix'), ("ké'oppe'", "oppe'", 'ké prefix'), ("kékekånno'", "kånno'", 'kéke prefix'), ("kéke'oppe'", "oppe'", 'kéke prefix'), ("na'kånno'", "kånno'", "na' prefix"), ("na'oppe'", "oppe'", "na' prefix"), ("muna'kånno'", "kånno'", "muna' prefix"), ("muna'oppe'", "oppe'", "muna' prefix"), ("nina'kånno'", "kånno'", "nina

## Apply Prefix Transformations - With Morphophonemic Changes

In this section we will apply prefixes that require changes to the base words. This includes the man- / fan- prefix which require the first letters of the base word to be changed. A group of other prefixes are also included for their vowel lowering changes on the base word.

### Man Prefix Rules

The Man- Prefix in Chamorro can be applied to a wide variety of words. But it is set apart from other prefixes because it requires morphophonemic changes to the base word. This section includes the lists of the different man- prefixes and the rules for morphophonemic changes.

In [33]:
# Create a dictionary of man- prefix rules
man_prefix_rules = {"b": "mam", "c": "mañ", "f": "mam", "g": "mang", "k": "mang", "p": "mam", "s": "mañ", "t": "man"}
fan_prefix_rules = {"b": "fam", "c": "fañ", "f": "fam", "g": "fang", "k": "fang", "p": "fam", "s": "fañ", "t": "fan"}

### Apply Man- Prefix Transformation

The Man- Prefix in Chamorro is distinguished by the need for consonant assimilation.

In [68]:
def apply_man_prefix(word, man_prefix_rules):
    """
    Transforms a word with the man- prefix and returns the transformed word
    """
    first_letter = word[0]
    man_prefix = man_prefix_rules[first_letter]
    
    if first_letter in man_prefix_rules:
        if first_letter == "c":
            new_word = man_prefix + word[2:]
        else:
            new_word = man_prefix + word[1:]
    else:
        new_word = "man" + word
    
    return new_word

In [70]:
### DELETE ME TEST CODE ###
word = "fa'tinas"
print(apply_man_prefix(word, man_prefix_rules))

mama'tinas


In [30]:
def apply_fan_prefix(word, fan_prefix_rules):
    """
    Transforms a word with the fan- prefix and returns the transformed word
    """
    return None

In [32]:
### DELETE ME TEST CODE ###

### Apply Other Prefix Transformations

Some prefixes require vowel lowering or vowel fronting to take place, which is similar to vowel harmony rules

In [100]:
### FIX ME: Confirm all the prefixes that require vowel-lowering ###
vowel_lowered_prefixes = ["gé", "ké"]

In [96]:
def apply_vowel_lowered_prefixes(data, prefixes):
    """
    Transforms words with prefixes that require vowel lowering.
    First we will transform the root word with vowel lowering
    Then we will attach the prefix
    """
    
    return None

In [27]:
### DELETE ME TEST CODE ###

# Apply Suffixes

**About Chamorro Suffixes:** Chamorro suffixes are attached to the end of a word, and sometimes they are preceded by a dash (-). All possessive pronouns are also included in this section as suffixes.

In [20]:
# Create possessive pronoun lists
possessive_pronouns_one_syllable = ["-hu", "-mu", "-ña", "-ta"]
possessive_pronouns_two_syllable = ["-ñiha", "-miyu", "-måmi", "-ñíñiha", "-mímiyu", "-måmåmi"]

# Create a list of other suffixes
### SUFFIX LIST GOES HERE ###

# Apply Circumfixes

**About Chamorro Circumfixes:** There is only one circumfix in Chamorro, with is the fan-...-an circumfix. Morphophonemic changes to the base word are usually required.

# Apply Reduplication

In this section, we will be experimenting with different ways to transform words with reduplication. There are several forms of reduplication in Chamorro, and we will start with the most common ones, which is to reduplicate on the stressed syllable. To achive this, we will first split a word into syllables and then count the syllables. In general, the second to last syllable is the one that is reduplicated. We do this because spelling in Chamorro is not consistent, and we cannot always rely upon stressed characters to be in our data.

In [159]:
# def split_syllables(word):
#     """
#     Splits a word into syllables in preparation for applying reduplication.
#     We reduplicate the consonant and the vowels attached to it
#     If the syllable ends in a consonant, don't repeat that last consonant
#     If the syllable has two vowels, only repeat the first vowel
#     But if they are vowel combos like ua, ue, ui, uo, we repeat the entire vowel combo
#     """
#     # Match consonant(s) + vowel(s) + optional consonant(s), including glottal stops and diphthongs
#     pattern = re.compile(r"([bcdfghjklmnpqrstvwxyz]*[aeiouåáéíóú]*['])", re.IGNORECASE)
#     syllables = pattern.findall(word)
#     syllable_count = len(syllables)-1
#     return (syllables, syllable_count)

# Export to CSV

We will take all the word lists from above and export them to CSV files.

In [92]:
# def convert_to_dataframe(affixed_words):
#     """
#     In this function, we convert our newly affixed words to a dataframe, and then export it to a CSV file.
#     We will also include metadata from the original word list to our exported CSV file.
#     """
#     # Get new words
#     new_words = affixed_words[0]
    
#     # Get old words
#     old_words = affixed_words[1]
    
#     # Get Affix Name
#     affix_name = affixed_words[0][0][2]
    
#     # Convert list to dataframe
#     infixed_words_df = pd.DataFrame(new_words, columns=["Word", "Term", "Affix"])
#     # Add the original Definition and Root Word to the infixed words df
#     filtered_df = old_words[["Term", "Definition", "Root Word"]]
#     infixed_words_df = pd.merge(infixed_words_df, filtered_df, on="Term", how="left")
    
#     # Save dataframe as CSV
#     infixed_words_df.to_csv(f"{affix_name}_affixed_words.csv", index=False, encoding="utf-8")

In [99]:
# # TEST Pass thru the convert_to_dataframe function
# infixed_words = [um_infixed_words, in_infixed_words]

# for output in infixed_words:
#     convert_to_dataframe(output)